In [9]:
import os
import glob
import subprocess
import itertools
from osgeo import gdal
from datetime import datetime

In [2]:
safes = glob.glob('*SAFE')
lines = [f'CALL Sen2Cor-02.09.00-win64\L2A_Process.bat {x}' for x in safes]
lines = ['@ECHO OFF'] + lines
with open('to_level2.bat', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')
# run to_level2.bat on command line

In [3]:
b08 = glob.glob('.\*\GRANULE\*\IMG_DATA\R10m\*B08_10m.jp2')
lines = [f'CALL rio convert {x} -f ENVI -o {os.path.basename(x).split(".")[0]}.dat' for x in b08]
lines = ['@ECHO OFF'] + lines
with open('to_envi.bat', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')
# run to_envi.bat on command line

In [4]:
b08dat = glob.glob('*b08_10m.dat')
# b08dat = ['T60HUB_20160206T221552_B08.jp2','T60HUB_20160306T221552_B08.jp2','T60HUB_20160406T221552_B08.jp2']

In [5]:
dates = [datetime.strptime(x[7:22],'%Y%m%dT%H%M%S') for x in b08dat]
file_dic = dict(zip(dates,b08dat))

combos = sorted(sorted(x) for x in itertools.combinations(dates,2))
files = [[file_dic[x[0]],file_dic[x[1]]] for x in combos]

# if using bounds:
#for bounds, sample=x, line=y
#xstart, xend, ystart, yend
# poroa1 = '1 10136 10838 9236 10480'
# poroa2 = '1 8900 10900 8500 10500'
# bounds = poroa2
# dic = {'primary':[f'{os.path.abspath(x[0])} {bounds}' for x in files],
#        'secondary':[f'{os.path.abspath(x[1])} {bounds}' for x in files],
#        'out_name':[os.path.join(os.getcwd(),f'cosicorr_{x[0][7:22]}_{x[1][7:22]}.dat') for x in files]}

# if not using bounds:
dic = {'primary':[f'{os.path.abspath(x[0])}' for x in files],
       'secondary':[f'{os.path.abspath(x[1])}' for x in files],
       'out_name':[os.path.join(os.getcwd(),f'cosicorr_{x[0][7:22]}_{x[1][7:22]}.dat') for x in files]}

for key in dic:
    with open(f'{key}.txt', 'w') as f:
        for line in dic[key]:
            f.write(line)
            f.write('\n')

In [6]:
# frequency, winMaxX, winMaxY, winMinX, winMinY, stepX, stepY, nbRobustIteration, MaskThreshold, resampling, grid
options = ['frequency 32 32 32 32 4 4 5 0.9 0 0'] * len(files)
with open('options.txt', 'w') as f:
    for line in options:
        f.write(line)
        f.write('\n')

In [15]:
# create separate vrt files for each band
files = glob.glob('*.dat')
files.sort()
for fname in files:
    for b, dataset in enumerate(['EWOffset','NSOffset','offsetSNR']):
        fname_out = f'{fname.split(".")[0]}_{dataset}.vrt'
        vrt_options = gdal.BuildVRTOptions(bandList=[b+1])
        my_vrt = gdal.BuildVRT(fname_out, fname, options=vrt_options)
        my_vrt = None

In [18]:
# write mintpy file
files = glob.glob('*.vrt')
files.sort()
lines = [f'{f} {f[9:17]} {f[25:33]}\n' for f in files]
with open('metadata.txt', 'w') as f:
    f.writelines(lines)